In [ ]:
import numpy as np
import tensorflow as tf
import os
import matplotlib.pyplot as plt
# import tensorflow_hub as hub
#os.environ['TF_CPP_MIN_LOG_LEVEL'] = 2

from PIL import Image
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense,Dropout,GlobalAveragePooling2D
from tensorflow.keras.preprocessing.image import img_to_array, load_img,ImageDataGenerator
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.losses import SparseCategoricalCrossentropy
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from tensorflow.keras.callbacks import EarlyStopping
from keras.api._v2.keras.callbacks import TensorBoard

from tqdm import tqdm

In [ ]:
import tensorflow.compat.v1 as tf
config = tf.compat.v1.ConfigProto()
config.gpu_options.allow_growth = True
# Create your session with this config


In [ ]:
class_labels = os.listdir('Images')
class_labels

In [ ]:
train_datagen = ImageDataGenerator(rescale = 1./255, validation_split = 0.2)
training_set = train_datagen.flow_from_directory('Images', 
                                                 target_size = (256, 256), 
                                                 batch_size = 16,
                                                 class_mode = 'categorical',
                                                 subset = "training",
                                                 shuffle = True)

test_set = train_datagen.flow_from_directory('Images',
                                            target_size = (256, 256),
                                            batch_size = 16,
                                            subset = "validation",
                                            class_mode = 'categorical',
                                            shuffle = False)

In [ ]:
type(training_set)

In [ ]:

# Load the EfficientNetV2-S base model (pre-trained on ImageNet)
base_model = tf.keras.applications.EfficientNetV2S(
    include_top=False,
    weights='imagenet',
    input_shape=(256, 256, 3),  # Adjust input shape as needed
    #include_preprocessing=True  # Keep preprocessing enabled
)

base_model.trainable = True
# Add a Global Average Pooling layer to reduce spatial dimensions
gap_layer = GlobalAveragePooling2D()(base_model.output)

# Create your custom model by adding additional layers
dense_layer1 = Dense(units=4096, activation='relu')(gap_layer)
dropout1 = Dropout(0.5)(dense_layer1)
dense_layer2 = Dense(units=4096, activation='relu')(dense_layer1)
dropout2 = Dropout(0.5)(dense_layer2)
output_layer = Dense(units=9, activation='softmax')(dropout2)

# Define the complete model
efficient_net = keras.Model(inputs=base_model.input, outputs=output_layer)

# Compile the model
efficient_net.compile(optimizer=Adam(), loss= 'categorical_crossentropy', metrics=['accuracy'])

# Print the model summary
efficient_net.summary()


In [ ]:
early_stopping = EarlyStopping(
    monitor='val_accuracy',  # Monitor validation loss
    patience=10,          # Number of epochs with no improvement after which training will be stopped
    restore_best_weights=True  # Restore model weights from the epoch with the best validation loss
)

tf_callbacks = tf.keras.callbacks.TensorBoard(
    log_dir='logs',
    histogram_freq=1,
    write_graph=True,
    write_images=False,
    write_steps_per_second=False,
    update_freq='epoch',
    profile_batch=0,
    embeddings_freq=0,
    embeddings_metadata=None
)

In [ ]:

history = efficient_net.fit(
    training_set,
    validation_data=test_set,
    callbacks=[early_stopping,tf_callbacks],
    epochs=100,# Set an arbitrary large number of epochs
    verbose = 1
)

In [ ]:
efficient_net.save('Large_fish_dataset.h5')
efficient_net.save_weights ('Large_fish_dataset.h5')

In [ ]:
#plot the training and validation accuracy and loss at each epoch
loss = history.history['loss']
val_loss = history.history['val_loss']
epochs = range(1, len(loss) + 1)
plt.plot(epochs, loss, 'y', label='Training loss')
plt.plot(epochs, val_loss, 'r', label='Validation loss')
plt.title('Training and validation loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()


acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

plt.plot(epochs, acc, 'y', label='Training acc')
plt.plot(epochs, val_acc, 'r', label='Validation acc')
plt.title('Training and validation accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.show()

In [ ]:
# Assuming `x_test` and `y_test` are your test data and labels
test_loss, test_accuracy = efficient_net.evaluate(test_set)
print(f"Test Loss: {test_loss}, Test Accuracy: {test_accuracy}")


In [ ]:
y_pred = efficient_net.predict(test_set)
y_pred

In [ ]:
y_pred_new = 1*(y_pred > 0.5)
y_pred_new

In [ ]:
y_test_set = np.concatenate([test_set.next()[1] for i in range(test_set.__len__())])
y_test_set = y_test_set.astype(int)
y_test_set = y_test_set.reshape(len(y_test_set), 9)
print(y_test_set)

In [ ]:
from sklearn.metrics import confusion_matrix,accuracy_score
import numpy as np

# Assuming y_test and y_pred are your true and predicted labels respectively
y_test_decoded = np.argmax(y_test_set, axis=1)
y_pred_decoded = np.argmax(y_pred_new, axis=1)

cm = confusion_matrix(y_test_decoded, y_pred_decoded)
acc= accuracy_score(y_test_set, y_pred_new)
print(acc,"\n")
print(cm)


In [ ]:
%tensorboard--logdir logs